# pandas 输入输出

In [44]:
import pandas as pd
import numpy as np

from io import StringIO

## CSV

### 数据交互

In [45]:
data = "col1,col2,col3\na,b,1\na,b,2\nc,d,3"
pd.read_csv(StringIO(data))

,col1,col2,col3
0,a,b,1
1,a,b,2
2,c,d,3


可以仅指定部分列：

In [46]:
pd.read_csv(StringIO(data), usecols=lambda x: x.upper() in ["COL1", "COL3"])

,col1,col3
0,a,1
1,a,2
2,c,3


也可以仅指定部分行：

In [47]:
pd.read_csv(StringIO(data), skiprows=lambda x: x % 2 != 0)

,col1,col2,col3
0,a,b,2


可以使用 `names` 参数指定列名。如果希望替换原字符流第一行，则设置 `header=0`。

In [48]:
data = "a,b,c\n1,2,3\n4,5,6\n7,8,9"
pd.read_csv(StringIO(data), names=["foo", "bar", "baz"], header=0)

,foo,bar,baz
0,1,2,3
1,4,5,6
2,7,8,9


如果第一行也是数据，列名需要手动添加，则设置 `header=None`。

In [49]:
pd.read_csv(StringIO(data), names=["foo", "bar", "baz"], header=None)

,foo,bar,baz
0,a,b,c
1,1,2,3
2,4,5,6
3,7,8,9


还可以实现跳过前面若干行的功能，例如跳过第一行：

In [50]:
data = "skip this skip it\na,b,c\n1,2,3\n4,5,6\n7,8,9"
pd.read_csv(StringIO(data), header=1)

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


使用 `dtype='category'`，生成的类别将始终被解析为字符串。

In [51]:
data = "col1,col2,col3\na,b,1\na,b,2\nc,d,3"
df = pd.read_csv(StringIO(data), dtype="category")
df.dtypes

col1    category
col2    category
col3    category
dtype: object

使用参数 `comment=#` 来忽略以 `#` 开头的注释。

In [52]:
data = "\na,b,c\n  \n# commented line\n1,2,3\n\n4,5,6"
pd.read_csv(StringIO(data), comment="#")

,a,b,c
0,1,2,3
1,4,5,6


使用参数 `skip_blank_lines` 来指定是否跳过空行，默认值为 `True`。

In [53]:
data = "a,b,c\n\n1,2,3\n\n\n4,5,6"
pd.read_csv(StringIO(data), skip_blank_lines=False)

,a,b,c
0,NaN,NaN,NaN
1,1.0,2.0,3.0
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,4.0,5.0,6.0


### 文件读写

In [54]:
df = pd.DataFrame(np.random.randint(0, 10, size=(3, 4)),
                  index=pd.date_range('2022/2/22', periods=3),
                  columns=list('ABCD'))
df.to_csv('dist/foo.csv', encoding='utf-8')

参数 `index_col=0` 指定 `csv` 文件的第一列为 `index`。

In [55]:
pd.read_csv('src/foo.csv', index_col=0)

,A,B,C,D
2022-02-22,9,4,9,2
2022-02-23,3,1,3,4
2022-02-24,5,9,8,7


## Excel

使用 **pandas** 读写 Excel 文件需要安装 `openpyxl` 包。

In [56]:
df = pd.DataFrame(np.random.randint(0, 10, size=(3, 4)),
                  index=pd.date_range('2022/2/22', periods=3),
                  columns=list('ABCD'))
df.to_excel('dist/foo.xlsx', sheet_name='Sheet1')

In [57]:
pd.read_excel('src/foo.xlsx', sheet_name='Sheet1', index_col=0, na_values=['NA'])

,A,B,C,D
2022-02-22,2,2,8,1
2022-02-23,0,6,4,2
2022-02-24,3,2,6,6


如果要读取一个 Excel 文件的多个 Sheet，使用 `ExcelFile` 对象。

In [58]:
data = {}

with pd.ExcelFile("src/foo.xlsx") as xlsx:
    data["Sheet1"] = pd.read_excel(xlsx, "Sheet1", index_col=0, na_values=['NA'])
    data["Sheet2"] = pd.read_excel(xlsx, "Sheet2", index_col=0, na_values=['NA'])

data

{'Sheet1':             A  B  C  D
 2022-02-22  2  2  8  1
 2022-02-23  0  6  4  2
 2022-02-24  3  2  6  6,
 'Sheet2':             A  B  C  D
 2022-02-22  1  2  4  3
 2022-02-23  3  2  1  2
 2022-02-24  3  1  4  6}